In [1]:
import os, os.path as op
import shutil
import yaml
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pandas as pd
import xarray as xr
from wand.image import Image as WImage

## Defining the home and data directories
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

# Verifying if RAiDER is installed correctly
try:
    import RAiDER
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

Tutorial directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_ARIA
Work directory:  /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_ARIA


### helpers

In [2]:
def new_yaml_group(example_yaml, dct_group, dst='tmp.yaml'):
    """ Write a temporary yaml file with the new 'value' for 'key', preserving parms in example_yaml"""
    with open(example_yaml, 'r') as f:
        try:
            params = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            raise ValueError(f'Something is wrong with the yaml file {example_yaml}')
    
    params = {**params, **dct_group}
    dst = op.join(os.path.dirname(example_yaml), dst)
#     print (params)
    
    with open(dst, 'w') as fh:
        yaml.dump(params, fh, default_flow_style=False)
    
    print ('Wrote new cfg file:', dst)
    return dst

#### Pieces of prepARIA

These are just for temporarily making the config file

In [7]:
def makeLatLonGrid(f:str, reg, out_dir):
    '''
    Convert the geocoded grids to lat/lon files for input to RAiDER
    '''
    from RAiDER.utilFcns import writeArrayToRaster
    group   = 'science/grids/data'
    lat_f   = os.path.join(f'NETCDF:"{f}":{group}/latitude')
    lon_f   = os.path.join(f'NETCDF:"{f}":{group}/longitude')


    ds   = xr.open_dataset(f, group='science/grids/data')

    gt   = (0, 1, 0, 0, 0, 1)
    proj = ds['crs'].crs_wkt

    lats = ds.latitude.data
    lons = ds.longitude.data

    ySize = len(lats)
    xSize = len(lons)

    # ISCE lats are ordered from smallest to biggest
    LATS  = np.flipud(np.tile(lats, (xSize, 1)).T)
    LONS  = np.tile(lons, (ySize, 1))

    dst_lat = op.join(out_dir, f'lat_{reg}.geo')
    dst_lon = op.join(out_dir, f'lon_{reg}.geo')

    writeArrayToRaster(LATS, dst_lat, 0., 'GTiff', proj, gt)
    writeArrayToRaster(LONS, dst_lon, 0., 'GTiff', proj, gt)

    logger.debug('Wrote: %s', dst_lat)
    logger.debug('Wrote: %s', dst_lon)
    return LATS, LONS

In [20]:
def parse_time_GUNW(f:str):
    """ Get the center time of the secondary date from the filename """
    tt = op.basename(f).split('-')[7]
    return f'{tt[:2]}:{tt[2:4]}:{tt[4:]}'

## Hampton Roads Test

In [15]:
yaml_base = op.join(work_dir, 'example_yamls', 'raider.yaml')
gunw = op.join(work_dir, 'data', 'S1-GUNW-A-R-004-tops-20181219_20181113-230629-37654N_35778N-PP-ede0-v2_0_2.nc')
orb1 = op.join(work_dir, 'orbits', 'S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF')
orb2 = op.join(work_dir, 'orbits', 'S1A_OPER_AUX_POEORB_OPOD_20190108T120818_V20181218T225942_20181220T005942.EOF')
makeLatLonGrid(gunw, 'HR', op.join(work_dir, 'data'));

Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_ARIA/data/lat_HR.geo
Wrote: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_ARIA/data/lon_HR.geo


In [ ]:
# first just use config file option

In [ ]:
# ISCE lat/lon files (Hampton Roads)
# this downloads the DEM and intersects it

grp = {'aoi_group': {'lat_file': f'{work_dir}/data/lat_HR.geo', 'lon_file': f'{work_dir}/data/lon_HR.geo'}, 
       'weather_model': 'ERA5',
       'date_group': {'date_start': '20181219'},
       'time_group': {'time': parse_time_GUNW(gunw)},
       'los_group' : {'orbit_file': {orb2}}
      }

cfg = new_yaml_group(yaml_base, grp)
!raider.py {cfg}

## store it for use later
# shutil.copy(f'{work_dir}/GLO30_fullres_dem.tif', f'{work_dir}/data/GLO30_fullres_dem_HR.tif')

Wrote new cfg file: /Users/buzzanga/Software_InSAR/RAiDER-docs_git/notebooks/RAiDER_ARIA/example_yamls/tmp.yaml
Starting to run the weather model calculation
Time: 20181219
Beginning weather model pre-processing
Extent of the weather model is (xmin, ymin, xmax, ymax):-78.93, 35.38, -74.93, 37.88
Extent of the input is (xmin, ymin, xmax, ymax): -78.53, 35.78, -75.26, 37.65
Output SNWE: [35.78, 37.660000000000004, -78.54, -75.24]
Output cube spacing: 0.02
Reading glo_30 Datasets: 100%|██████████████████| 25/25 [00:44<00:00,  1.78s/it]


## LA Test